In [ ]:
!pip install telethon requests qrcode Pillow nest_asyncio opencv-python


In [10]:
# Installation of necessary libraries
!pip install telethon requests qrcode Pillow nest_asyncio opencv-python

# Import libraries
import os
import json
import requests
import asyncio
import qrcode
from telethon import TelegramClient
import nest_asyncio
from google.colab.patches import cv2_imshow
import cv2

# Apply nest_asyncio for correct operation in Google Colab
nest_asyncio.apply()

# --- Your data (be sure to replace!) ---
api_id = os.environ['API_ID']
api_hash = 'os.environ['API_HASH']'
phone = 'os.environ['PHONE_NUMBER']'

# n8n webhook URL
n8n_webhook_url = 'os.environ['N8N_WEBHOOK_URL']'

# --- Список каналов для парсинга ---
channel_names = ['@BlackAudit','@violavolkova']
# ---------------------------------------------

async def main():
    # Подключение к Telegram
    client = TelegramClient('my_session', api_id, api_hash)
    await client.connect()

    # Проверка авторизации и логин по QR-коду, если не авторизован
    if not await client.is_user_authorized():
        try:
            # Генерация QR-кода
            qr_login = await client.qr_login()
            url = qr_login.url
            qr = qrcode.QRCode()
            qr.add_data(url)
            qr_img = qr.make_image(fill_color="black", back_color="white")
            qr_img.save("qrcode.png")
            img = cv2.imread("qrcode.png", cv2.IMREAD_UNCHANGED)
            cv2_imshow(img)
            print('Отсканируйте QR-код для входа')
            await client.run_until_disconnected()
        except Exception as e:
            print(f"Ошибка авторизации: {e}")
            return

    # Получение последних 5 сообщений из каждого канала в списке
    results = []
    for channel_name in channel_names:
        async for msg in client.iter_messages(channel_name, limit=5):
            if msg.text:
                results.append({
                    'channel': channel_name, # Добавлено название канала
                    'id': msg.id,
                    'text': msg.text,
                    'date': str(msg.date)
                })

    # Отправка данных в n8n
    try:
        requests.post(n8n_webhook_url, json=results)
        print("Данные успешно отправлены в n8n.")
    except Exception as e:
        print(f"Ошибка при отправке данных в n8n: {e}")

    # Отключение от Telegram
    await client.disconnect()

if __name__ == '__main__':
    asyncio.run(main())

Данные успешно отправлены в n8n.
